In [ ]:
!jupyter-nbconvert SQLAlchemy_for_chemists.ipynb --to slides --post serve

Introduction to SQLAlchemy and Jupyter Notebooks
==========================================

### John Hey

#### Adapted from material by:
- Lee Bergstrand
- Matt McInnes

**Note:** This tutorial is based off the [official SQLAlchemy ORM tutorial](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html). 

Introduction to SQLAlchemy for Chemists
==========================

Firstly we need to import sqlalchemy and begin creating our database:

In [ ]:
import sqlalchemy

# Connect to the database...
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=False)

**Note** that the string ``` 'sqlite:///:memory:' ``` sets the backend to sqlite and creates a database in memory. (RAM) 

This is useful for testing but we can change it to save to the harddisk later if we want our data to be permenant.

### Creating Database-Backed Classes
---

Our next step is to define our classes which will be mapped onto tables within our database. 

We do this by defining a class in the normal pythonic way but with some extra SQLAlchemy commands. 

SQLAlchemy will then use this class to define and create the relavent tables.  

In SQLAlchemy the ORM utilizes classes inheriting from the SQLAlchemy base class. 

**Note:** ORMs (Object Relational Mapper) are classes which allow you to refer to data stored in your database from another language.

Columns within a database table can usually only store scalar data (Ints, Strings etc.) an ORM allows you to map more complex datatypes onto these columns within your database. 

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# In our example we are going to create 

Classes define RDBMS (Relational DataBase Management Scheme) table attributes:

- We must define the `__tablename__` attribute
- Then we define some `Column` attributes
>- Columns have datatypes (Integer, Float, etc.)
>- Columns have constraints (Primary Key, Foreign Key, etc.)

In [ ]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    # Defines to_string() representation 
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
                self.name, self.fullname, self.password)

SQLAlchemy will set up your tables for you...

In [ ]:
Base.metadata.create_all(engine)

##### Issued SQL:

```SQL
CREATE TABLE users (
    id INTEGER NOT NULL, name VARCHAR,
    fullname VARCHAR,
    password VARCHAR,
    PRIMARY KEY (id)
```

### Creating a Session

---

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

### Adding, Updating and Commiting Objects
---

Once you have your tables create you can then start creating objects.

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)
ed_user

When you do a query SQLAlchemy, your uncommited changes are flushed to the db (within a transaction). SQLAlchemy then requeries the database in order to gain the value of the objects primary key column.

In [ ]:
our_user = session.query(User).filter_by(name='ed').first()

##### Issued SQL:

*Insert (Flush)*

```SQL
INSERT INTO users (name, fullname, password) VALUES ('ed', 'Ed Jones', 'edspassword')
```

*Query Select*

```SQL
SELECT users.id AS users_id,
        users.name AS users_name,
        users.fullname AS users_fullname,
        users.password AS users_password
FROM users
WHERE users.name = 'ed'
 LIMIT 1 OFFSET 0
```

Since both of `end_user` and `our_user` have the same primary key the are mapped to the same object.

In [ ]:
ed_user is our_user

Multiple objects can be added to the session and objects can be modified.

In [ ]:
session.add_all([
        User(name='wendy', fullname='Wendy Williams', password='foobar'),
        User(name='mary', fullname='Mary Contrary', password='xxg527'),
        User(name='fred', fullname='Fred Flinstone', password='blah')])

ed_user.password = 'f8s7ccs'

Finally, the transaction can be commited. The remaining changes are flushed to the database.

In [ ]:
session.commit()

##### Issued SQL:

*Flush remaining changes (inserts and updates)*

```SQL
UPDATE users SET password='f8s7ccs' WHERE users.id = 1

INSERT INTO users (name, fullname, password) VALUES ('wendy', 'Wendy Williams', 'foobar')
INSERT INTO users (name, fullname, password) VALUES ('mary', 'Mary Contrary', 'xxg527')
INSERT INTO users (name, fullname, password) VALUES (fred', 'Fred Flinstone', 'blah')
```

*Finally, commit the changes*

```SQL
COMMIT
```

### Rolling Back
---

In SQLAlchemy all database operations are performed within a transaction. 

For example we can edit and query for objects:

In [ ]:
ed_user.name = 'Edwardo'

fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

We can do a rollback and our changes are reset.

In [ ]:
session.rollback()

##### Issued SQL:

```SQL
ROLLBACK
```

Our objects are reset to their pervious states and our uncommited objects are removed.

In [ ]:
ed_user.name

In [ ]:
fake_user in session

### Querying
---

In [ ]:
for instance in session.query(User).order_by(User.id):
    print(instance)

##### Issued SQL:

```SQL
SELECT users.id AS users_id,
        users.name AS users_name,
        users.fullname AS users_fullname,
        users.password AS users_password
FROM users ORDER BY users.id
```

In [ ]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

##### Issued SQL:

```SQL
SELECT users.name AS users_name,
        users.fullname AS users_fullname
FROM users
```

### Common Filtering Operators

Here’s an overview of some common operators used by the `filter()` fucntion:

#### equals:

In [ ]:
for user in session.query(User).filter(User.name == 'ed'):
    print(user)

#### not equals:

In [ ]:
for user in session.query(User).filter(User.name != 'ed'):
    print(user)

#### LIKE:

In [ ]:
for user in session.query(User).filter(User.name.like('%ed%')):
    print(user)

#### IN:

In [ ]:
for user in session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)

#### NOT IN:

In [ ]:
for user in session.query(User).filter(~User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)

#### AND:

In [ ]:
for user in session.query(User).filter(User.name == 'ed', User.fullname == 'Ed Jones'):
    print(user)

#### OR:

In [ ]:
from sqlalchemy import or_
for user in session.query(User).filter(or_(User.name == 'ed', User.name == 'wendy')):
    print(user)

### Relationships
---

Multiple classes can be created and linked together using class relations.

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

Once primary and foreign keys are established, classes can be connected together via relationships.

In [ ]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

Base.metadata.create_all(engine) # Flush schema changes to the DBMS.

In the above mappings:
- ```user.addresses``` maps to a user's list of address objects.
- ```address.user``` maps to an address's parent user object.

We create a "Jack" object and it initially has no addresses.

In [ ]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses

We then add some address objects.

In [ ]:
jack.addresses = [Address(email_address='jack@google.com'), Address(email_address='j25@yahoo.com')]

In [ ]:
jack.addresses[1]

We see that the "Jack" object's first address's user parameter refers back to the "Jack" object.

In [ ]:
jack.addresses[1].user

When we add "Jack" to the session and commit, the two address objects of "Jack" are also committed in a cascading fashion.

In [ ]:
session.add(jack)

session.commit()

jack = session.query(User).filter_by(name='jack').one()
jack.addresses